In [27]:
import pints
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [29]:
import pints.toy as toy

class RescaledModel(pints.ForwardModel):
    def __init__(self):
        self.base_model = toy.LogisticModel()
    
    def simulate(self, parameters, times):
        # Run a simulation with the given parameters for the
        # given times and return the simulated values
        r, k = parameters
        r = r / 50
        k = k * 500
        return self.base_model.simulate([r, k], times)
    
    def simulateS1(self, parameters, times):
        # Run a simulation with the given parameters for the
        # given times and return the simulated values
        r, k = parameters
        r = r / 50
        k = k * 500
        return self.base_model.simulateS1([r, k], times)
    
    def n_parameters(self):
        # Return the dimension of the parameter vector
        return 2
    
model = toy.LogisticModel()
true_parameters = [0.015, 500]
start_parameters = [0.75, 1.0] # rescaled true parameters
times = np.linspace(0, 1000, 400)

In [30]:
# Simulate time series
org_values = model.simulate(true_parameters, times)
range_values = max(org_values) - min(org_values)
noise = 0.05 * range_values
values = org_values + np.random.normal(0, noise, org_values.shape)

model = RescaledModel()
problem = pints.SingleOutputProblem(model, times, values)
log_likelihood = pints.GaussianKnownSigmaLogLikelihood(problem, noise)

# Create (rescaled) bounds for our parameters and get prior
bounds = pints.RectangularBoundaries([0.5, 0.8], [1.0, 1.2])
log_prior = pints.UniformLogPrior(bounds)
# Create a posterior log-likelihood (log(likelihood * prior))
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Train emulator
input_parameters = log_prior.sample(2000)
likelihoods = np.apply_along_axis(log_likelihood, 1, input_parameters)

#likelihoods = np.array([0.7*l for l in likelihoods])
X_train, X_valid, y_train, y_valid = train_test_split(input_parameters, likelihoods, test_size=0.3, random_state=0)
emu = pints.MultiLayerNN(problem, X_train, y_train, input_scaler=MinMaxScaler(), output_scaler=StandardScaler())
emu.set_parameters(layers=6, neurons=64, hidden_activation='relu', activation='linear', learning_rate=0.0001)
hist = emu.fit(epochs=500, batch_size=32, X_val=X_valid, y_val=y_valid, verbose=0)
log_posterior_emu = pints.LogPosterior(emu, log_prior)

In [158]:
p1_range = np.linspace(0.5, 1.0, 100)
p2_range = np.linspace(0.8, 1.2, 100)
p1_grid, p2_grid = np.meshgrid(p1_range, p2_range)
grids = [p1_grid, p2_grid]
grid = np.dstack(tuple(grids))
rows, cols, n_params = grid.shape
flattened_grid = grid.reshape((rows * cols, n_params))
grid

array([[[0.5       , 0.8       ],
        [0.50505051, 0.8       ],
        [0.51010101, 0.8       ],
        ...,
        [0.98989899, 0.8       ],
        [0.99494949, 0.8       ],
        [1.        , 0.8       ]],

       [[0.5       , 0.8040404 ],
        [0.50505051, 0.8040404 ],
        [0.51010101, 0.8040404 ],
        ...,
        [0.98989899, 0.8040404 ],
        [0.99494949, 0.8040404 ],
        [1.        , 0.8040404 ]],

       [[0.5       , 0.80808081],
        [0.50505051, 0.80808081],
        [0.51010101, 0.80808081],
        ...,
        [0.98989899, 0.80808081],
        [0.99494949, 0.80808081],
        [1.        , 0.80808081]],

       ...,

       [[0.5       , 1.19191919],
        [0.50505051, 1.19191919],
        [0.51010101, 1.19191919],
        ...,
        [0.98989899, 1.19191919],
        [0.99494949, 1.19191919],
        [1.        , 1.19191919]],

       [[0.5       , 1.1959596 ],
        [0.50505051, 1.1959596 ],
        [0.51010101, 1.1959596 ],
        .

In [124]:
dx = p1_grid[0][1]-p1_grid[0][0]
dy = p2_grid[1][0]-p2_grid[0][0]
dy

0.004040404040404066

In [54]:
test_splits = 50 # number of splits along each axis
r_grid, k_grid, test_data = pints.generate_grid(bounds.lower(), bounds.upper(), test_splits)

In [142]:
emu_prediction = np.apply_along_axis(emu, 2, grid)
model_prediction = np.apply_along_axis(log_likelihood, 2, grid)

In [153]:
Gx_emu, Gy_emu = np.gradient(emu_prediction.reshape(100,100), dx, dy)
G_model = np.gradient(model_prediction.reshape(100,100), dx, dy)

In [160]:
emu_prediction.reshape(100,100)

array([[-7938.2324, -7869.6855, -7790.861 , ..., -3982.9553, -4007.7324,
        -4032.5088],
       [-7977.3765, -7908.892 , -7829.258 , ..., -3947.212 , -3971.9893,
        -3998.4885],
       [-8018.667 , -7951.0337, -7871.4653, ..., -3911.167 , -3936.5955,
        -3965.363 ],
       ...,
       [-6659.4253, -6475.9126, -6287.052 , ..., -6063.48  , -6104.459 ,
        -6145.436 ],
       [-6659.6416, -6477.2915, -6290.628 , ..., -6101.68  , -6142.7275,
        -6183.538 ],
       [-6659.856 , -6479.875 , -6296.161 , ..., -6139.4507, -6180.289 ,
        -6220.322 ]], dtype=float32)

In [172]:
(emu([0.50505051, 0.8       ]) - emu([0.5       , 0.8       ])) / (0.50505051-0.5)

array([[13572.268]], dtype=float32)

In [168]:
(emu_prediction.reshape(100,100)[0][1]-emu_prediction.reshape(100,100)[0][0])/dx

13572.281249999913

In [174]:
Gx_emu

array([[-7750.5205  , -7762.8955  , -7602.6006  , ...,  7077.195   ,
         7077.1465  ,  6736.012   ],
       [-7963.0225  , -8053.4663  , -7979.8447  , ...,  7107.045   ,
         7042.5596  ,  6647.4297  ],
       [-8548.177   , -8572.25    , -8521.154   , ...,  7182.8174  ,
         6976.8896  ,  6602.9087  ],
       ...,
       [  -47.904785,  -272.87842 ,  -694.3535  , ..., -7715.2324  ,
        -7647.8467  , -7624.982   ],
       [  -42.635742,  -392.27783 ,  -901.8281  , ..., -7521.0996  ,
        -7507.1777  , -7413.6885  ],
       [  -42.442383,  -511.53223 , -1095.5742  , ..., -7478.5605  ,
        -7437.1816  , -7283.171   ]], dtype=float32)

In [177]:
(emu([0.5       , 0.8040404 ]) - emu([0.5       , 0.8       ])) / (0.8040404-0.8)

array([[-9688.16]], dtype=float32)

In [156]:
Gy_emu

array([[ 16965.352 ,  18237.232 ,  20082.062 , ...,  -6144.8096,
         -6132.241 ,  -6132.1504],
       [ 16949.883 ,  18329.682 ,  20723.11  , ...,  -6175.113 ,
         -6345.4805,  -6558.5684],
       [ 16739.24  ,  18216.205 ,  21277.084 , ...,  -6351.674 ,
         -6706.7603,  -7119.975 ],
       ...,
       [ 45419.39  ,  46081.223 ,  46043.7   , ..., -10281.159 ,
        -10142.062 , -10141.819 ],
       [ 45131.65  ,  45665.44  ,  45435.766 , ..., -10188.951 ,
        -10129.916 , -10100.61  ],
       [ 44545.285 ,  45007.234 ,  44976.9   , ..., -10112.574 ,
        -10007.798 ,  -9908.097 ]], dtype=float32)

In [89]:
from numpy import gradient
dx = np.cumsum(np.ones(5))
dx_uneven = [1., 2., 5., 9., 11.]
f_2d = np.arange(25).reshape(5, 5)

# distances must be scalars or have size equal to gradient[axis]
one = gradient(np.arange(5), 3.)
two = gradient(np.arange(5), np.array(3.))
three = gradient(np.arange(5), dx)
# dy is set equal to dx because scalar
four = gradient(f_2d, 1.5)
five = gradient(f_2d, np.array(1.5))

six = gradient(f_2d, dx_uneven, dx_uneven)
# mix between even and uneven spaces and
# mix between scalar and vector
seven = gradient(f_2d, dx, 2)

# 2D but axis specified
eight = gradient(f_2d, dx, axis=1)

In [91]:
f_2d

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [101]:
four

[array([[3.33333333, 3.33333333, 3.33333333, 3.33333333, 3.33333333],
        [3.33333333, 3.33333333, 3.33333333, 3.33333333, 3.33333333],
        [3.33333333, 3.33333333, 3.33333333, 3.33333333, 3.33333333],
        [3.33333333, 3.33333333, 3.33333333, 3.33333333, 3.33333333],
        [3.33333333, 3.33333333, 3.33333333, 3.33333333, 3.33333333]]),
 array([[0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667],
        [0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667],
        [0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667],
        [0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667],
        [0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667]])]

In [102]:
five

[array([[3.33333333, 3.33333333, 3.33333333, 3.33333333, 3.33333333],
        [3.33333333, 3.33333333, 3.33333333, 3.33333333, 3.33333333],
        [3.33333333, 3.33333333, 3.33333333, 3.33333333, 3.33333333],
        [3.33333333, 3.33333333, 3.33333333, 3.33333333, 3.33333333],
        [3.33333333, 3.33333333, 3.33333333, 3.33333333, 3.33333333]]),
 array([[0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667],
        [0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667],
        [0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667],
        [0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667],
        [0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667]])]

In [103]:
six

[array([[5.        , 5.        , 5.        , 5.        , 5.        ],
        [4.16666667, 4.16666667, 4.16666667, 4.16666667, 4.16666667],
        [1.48809524, 1.48809524, 1.48809524, 1.48809524, 1.48809524],
        [2.08333333, 2.08333333, 2.08333333, 2.08333333, 2.08333333],
        [2.5       , 2.5       , 2.5       , 2.5       , 2.5       ]]),
 array([[1.        , 0.83333333, 0.29761905, 0.41666667, 0.5       ],
        [1.        , 0.83333333, 0.29761905, 0.41666667, 0.5       ],
        [1.        , 0.83333333, 0.29761905, 0.41666667, 0.5       ],
        [1.        , 0.83333333, 0.29761905, 0.41666667, 0.5       ],
        [1.        , 0.83333333, 0.29761905, 0.41666667, 0.5       ]])]

In [104]:
seven

[array([[5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.]]),
 array([[0.5, 0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5, 0.5]])]

In [105]:
eight

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])